In [1]:
import time
import pandas as pd
import numpy as np
import math
from ortools.linear_solver import pywraplp


In [2]:
AllData = pd.ExcelFile("Input Data.xlsx")
StudentCourseData = pd.read_excel(AllData, 'StudentCourse')
TeacherCourseData = pd.read_excel(AllData, 'TeacherCourse')
TeacherBlockData = pd.read_excel(AllData, 'TeacherDay')

teacher_day = TeacherBlockData.copy().applymap(lambda x: 0 if x == 'N' else 1)
teacher_course = TeacherCourseData.copy().applymap(lambda x: 1 if x == 'Y' else 0)


In [3]:
# StudentCourseData['CS 5010 Programming Design Paradigm (Core)'][1] = 5 means that Student 1 wants to take Course 
# CS 5010 Programming Design Paradigm (Core) with a preference of 5
print(StudentCourseData['CS 5010 Programming Design Paradigm (Core)'][1])

# TeacherCourseData['CS 5010 Programming Design Paradigm (Core)'][5] ='Y' means that Course 
# CS 5010 Programming Design Paradigm (Core) can be taught by Teacher 3
print(TeacherCourseData['CS 5010 Programming Design Paradigm (Core)'][3])

# TeacherDayData[1][5] ='N' means that Day 'Mon' cannot be assigned to Teacher T5
print(TeacherBlockData['Mon'][5])

5
Y
N


In [4]:
numStudents = StudentCourseData.shape[0]
numTeachers = TeacherCourseData.shape[0]
numCourses = StudentCourseData.shape[1] - 1
numDays = TeacherBlockData.shape[1] - 1

numSem = 4
SeatCountPerCourse = 20
maxNumberOfCourses = 8

allStudents = range(numStudents)
allTeachers = range(numTeachers)
allCourses = range(numCourses)
allDays = range(numDays)
allSem = range(numSem)

StudentList = np.array(StudentCourseData['Name'])
TeacherList = np.array(TeacherCourseData['Name'])
CourseList = np.delete(np.array(TeacherCourseData.columns),0)
DayList = np.delete(np.array(TeacherBlockData.columns),0)
SemList = np.array(['Sem1','Sem2','Sem3','Sem4'])

allCoreCourses = [0,1,2,3]
allElectiveCourses = np.arange(4,numCourses)
print(CourseList)

['CS 5010 Programming Design Paradigm (Core)'
 'CS 5500 Foundations of Software Engineering (Core)'
 'CS 5600 Computer System (Core)' 'CS 5800 Algorithms (Core)'
 'CS 5100 Foundations of Artificial Intelligence'
 'CS 5335 Robotic Science and Systems'
 'CS 6120 Natural Language Processing' 'CS 6140 Machine Learning'
 'CS 7140 Advanced Machine Learning'
 'CS 7180 Special Topics in Artificial Intelligence'
 'CS 5340 Computer/Human Interaction' 'CS 6350 Empirical Research Methods'
 'CS 5520 Mobile Application Development' 'CS 5610 Web Development'
 'CS 6510 Advanced Software Development'
 'CS 6650 Building Scalable Distributed Systems'
 'CS 7580 Special Topics in Software Engineering'
 'CS 5200 Database Management Systems' 'CS 6200 Information Retrieval'
 'CS 6220 Data Mining Techniques'
 'CS 6240 Large-Scale Parallel Data Processing' 'CS 7150 Deep Learning'
 'CS 7280 Special Topics in Database Management'
 'CS 7290 Special Topics in Data Science'
 'CS 7295 Special Topics in Data Visualiza

In [5]:
solver = pywraplp.Solver('Timetabling Problem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

start_time = time.time()

# Define our binary variables for the students and teachers
X = {}
for s in allStudents:
    for c in allCourses:
        for sem in allSem:
            X[s,c,sem] = solver.BoolVar('X[%i,%i,%i]' % (s,c,sem))

Y = {}
for t in allTeachers:
    for c in allCourses:
        for d in allDays:
            for sem in allSem:
                Y[t,c,d,sem] = solver.BoolVar('Y[%i,%i,%i,%i]' % (t,c,d,sem))

# Define our objective function
solver.Maximize(solver.Sum(X[s,c,sem] * StudentCourseData[CourseList[c]][s] for s in allStudents for c in allCourses for sem in allSem))

# Each student must take two course on each sem
for s in allStudents:
    for sem in allSem:
        solver.Add(solver.Sum([X[s,c,sem] for c in allCourses]) == 2)

# Student can take courses only when it is available
for sem in allSem:
    for c in allCourses:
        for s in allStudents:
            solver.Add(solver.Sum([X[s, c, sem]]) <= 
                       solver.Sum([Y[t, c, d, sem] for t in allTeachers for d in allDays]))

# No teacher may teach more than two courses per sem
for sem in allSem:
    for t in allTeachers:
        solver.Add(solver.Sum([Y[t,c,d,sem] for c in allCourses for d in allDays]) <= 2)

# Atmost 2 courses are taught each day
for sem in allSem:
    for d in allDays:
        solver.Add(solver.Sum([Y[t,c,d,sem] for t in allTeachers for c in allCourses]) <= 2)

# Teacher can teach course only on the days they are available.
for sem in allSem:
    for t in allTeachers:
        for d in allDays:
            solver.Add(solver.Sum([Y[t, c, d, sem] for c in allCourses]) <= teacher_day[DayList[d]][t])

# Course if available in a sem should be taught on a single day
for sem in allSem:
    for c in allCourses:
        solver.Add(solver.Sum([Y[t, c, d, sem] for t in allTeachers for d in allDays]) <= 1)

# Teacher teaches course they perfer to teach
for sem in allSem:
    for t in allTeachers:
        for c in allCourses:
            solver.Add(solver.Sum([Y[t, c, d, sem] for d in allDays]) <= teacher_course[CourseList[c]][t])

# Total number of students enrolled in course should be less than total seats.
for sem in allSem:
    for c in allCourses:
        solver.Add(solver.Sum([X[s, c, sem] for s in allStudents]) <= SeatCountPerCourse)

# Student will take a course excatly once across 4 semesters
for s in allStudents:
    for c in allCourses:
        solver.Add(solver.Sum([X[s,c,sem] for sem in allSem]) <= 1)
        
# Each course is assigned to one teacher.
for sem in allSem:
    for c in allCourses:
        solver.Add(solver.Sum([Y[t,c,d,sem] for t in allTeachers for d in allDays]) <= 1)
        
# Total number of courses offered should be less than maximum courses campus can schedule in a semester. 
for sem in allSem:
    solver.Add(solver.Sum([Y[t, c, d, sem] for t in allTeachers for c in allCourses for d in allDays]) <= maxNumberOfCourses)
    
# At least one core course should be offered every semester 
for sem in allSem:
    solver.Add(solver.Sum([Y[t, c, d, sem] for t in allTeachers for c in allCoreCourses for d in allDays]) >= 1)

# At least one elective course should be offered every semester 
for sem in allSem:
    solver.Add(solver.Sum([Y[t, c, d, sem] for t in allTeachers for c in allElectiveCourses for d in allDays]) >= 1)

# Student must take CS5010 PDP course
for s in allStudents:
    solver.Add(solver.Sum([X[s,0,sem] for sem in allSem]) == 1)

# Student must take CS5800 Algo course
for s in allStudents:
    solver.Add(solver.Sum([X[s,3,sem] for sem in allSem]) == 1)

# Student must take CS5500/CS5600 course
for s in allStudents:
    solver.Add(solver.Sum([X[s,1,sem] + X[s,2,sem] for sem in allSem]) == 1)

current_time = time.time() 
reading_time = current_time - start_time         
sol = solver.Solve()
solving_time = time.time() - current_time

print('Optimization Complete with Total Happiness Score of', round(solver.Objective().Value()))
print("")
print('Our program needed', round(solving_time,3), 
      'seconds to determine the optimal solution')

Optimization Complete with Total Happiness Score of 1886

Our program needed 99.972 seconds to determine the optimal solution


In [6]:
timetable = []
for sem in allSem:
    semlist = []
    for d in allDays:
        daylist = ""
        for t in allTeachers:
            for c in allCourses:
                if Y[t,c,d,sem].solution_value() == 1:
                    daylist = daylist + CourseList[c] + " - " + TeacherList[t] + "\n"
        semlist.append(daylist.strip())
    timetable.append(semlist)
coursett = pd.DataFrame(timetable, columns = DayList, index = SemList)  

In [7]:
timetable = []
for s in allStudents:
    semlist = []
    for sem in allSem:
        courselist = ""
        for c in allCourses:
            if X[s,c,sem].solution_value() == 1:
                courselist = courselist + CourseList[c] + "\n"
        semlist.append(courselist.strip())
    timetable.append(semlist)
studenttt = pd.DataFrame(timetable, columns = SemList, index = StudentList)

In [8]:
with pd.ExcelWriter('output.xlsx') as writer:  
    coursett.to_excel(writer, sheet_name='Course Timetable')
    studenttt.to_excel(writer, sheet_name='student TimeTable')